In [1]:
from utils.chat import OllamaChat
from utils.prompts import get_persona
from company_about_summarizer import summarize_company_about
from utils.example_company.company_about import company_data
from utils.example_company.example_customer import example_customer
from utils.example_company.products_data import Products_data
import json

# Summarizing the Company Data

In [6]:
company_data_summarized=summarize_company_about(company_data)


In [7]:
system_prompt = get_persona("general_sales_agent",company_data=company_data_summarized)

In [20]:
chat = OllamaChat(
        model='llama3.1',  
        system_prompt=system_prompt,
        n_ctx=8000,
        temperature=1.2
    )

# Greeting the Customer

In [11]:
result=chat.chat(user_prompt="hello ?")


In [12]:
print(result)

Hello! My name is Alex, and I'm a representative from TechCare AI. I hope I'm catching you at a good time? I was calling to see if we could have a quick conversation about how TechCare AI's solutions might be able to support your business needs. Would now be a convenient time for you to chat, or would you prefer to schedule something for later?


# Shifting to Pitching Now

In [13]:
name=example_customer["customer_name"]
company=example_customer["customer_company"]
description=example_customer["company_description"]


product_service_details=json.dumps(Products_data["products_and_services"][0])



In [21]:
chat.system_prompt=get_persona("product_pitch_agent",customer_name=name,customer_company=company,company_description=description,product_service_details=product_service_details)

In [17]:
result=chat.chat(user_prompt="Yeah sure")

In [18]:
print(result)

So, John from Tech Innovators Inc., I've taken a look at your company profile, and it seems like you're a leader in providing cloud-based software solutions that help businesses streamline operations and enhance productivity. That's really impressive!

I'd like to discuss with you a product that we think could be particularly relevant to your business goals. Have you considered implementing an intelligent ticket management system to optimize customer service operations? Our ServiceFlow AI is designed to do just that, using AI-driven technology to categorize, route, and respond to tickets in a highly efficient manner.

With ServiceFlow AI, our customers have seen some really impressive results – such as a 98% accuracy rate in ticket categorization, which significantly reduces misrouting. We've also got pre-built integrations with leading CRMs like Salesforce, Zendesk, and HubSpot, making it easy to get started and scale up.

Would you like me to walk you through some specific examples o

In [22]:
result=chat.chat(user_prompt="tell me more about it how exactly it works ?")
print(result)

I'm glad you're interested in learning more.

So, ServiceFlow AI is built on a combination of advanced machine learning algorithms and robust integration capabilities. Here's a high-level overview of how it works:

1. **Ticket Ingestion**: Your customer service tickets are fed into the system via our pre-built integrations with popular CRMs like Salesforce, Zendesk, or HubSpot.
2. **Categorization**: The AI engine analyzes each ticket and assigns a category or tag based on natural language processing (NLP). This ensures that tickets are routed to the right teams for resolution, reducing misrouting and associated delays.
3. **Routing**: ServiceFlow's routing logic uses agent expertise, availability, and workload data to assign tickets to the best-suited team member. You can also customize routing rules based on your specific business needs.
4. **Automated Responses**: For common queries or frequently asked questions (FAQs), the AI system generates pre-built responses that are sent autom